Aplicando restrições na busca
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0.01 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos o valor zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



> <hr>

## Importações



In [1]:
import random

from funcoes import computa_mochila
from funcoes import funcao_objetivo_pop_mochila
from funcoes import populacao_cb as cria_populacao_inicial
from funcoes import selecao_roleta_maxima as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cb as funcao_mutacao

## Códigos e discussão



In [15]:
### CONSTANTES

# relacionadas à busca
# podem ser alteradas
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

# relacionadas ao problema a ser resolvido
# não podem ser alteradas sem prejudicar o experimento
LIMITE_DE_PESO = 15 # limitação
OBJETOS = {
    # dicionário baseado no que vocês enviaram na aula de Lógica
    # é um dicionário de dicionários

    "vinil falsificado da volta do One Direction": {
        "peso": 2,
        "valor": 2500,
    },
    "Harry Potter: ele voltou, confia!": {
        "peso": 3,
        "valor": 1500,
    },
    "Quadrinho super raro do Aranha-Homem da vida real": {
        "peso": 3,
        "valor": 7000,
    },
    "mesa dobrável para laptop": {
        "peso": 3,
        "valor": 150,
    },
    "tablet": {
        "peso": 0.6,
        "valor": 2400,
    },
    "teclado musical": {
        "peso": 3.5,
        "valor": 3000,
    },
    "bicicleta": {
        "peso": 16,
        "valor": 1000,
    },
    "lições em dia": {
        "peso": 8,
        "valor": 5000,
    },
    "energético": {
        "peso": 2,
        "valor": 1500,
    },
    "docinhos para o stress": {
        "peso": 5,
        "valor": 3000,
    },
}
NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys())) # nomes dos objetos em ordem alfabética dentro da lista

In [16]:
# Funções locais
# serão utilizadas somente no contexto do presente notebook

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_mochila(
        populacao, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES
    )

In [17]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

# variaveis para o hall da fama
    # reúne os melhores indivíduos
    # o hall da fama vai considerar então o valor do fitness (busca pelo maior) e a posição do indivíduo na população
melhor_fitness_ja_visto = -float("inf") # qualquer valor é maior do que menos infinito
melhor_individuo_ja_visto = [0] * NUM_OBJETOS # o melhor individuo estará na posição zero com o número de onjetos/genes definido na constante

for n in range(NUM_GERACOES): # para cada geração dentro do total de gerações

    # Seleção
    # selecionando os indivíduos com base nos fitness
    fitness = funcao_objetivo_pop(populacao) # calculo do fitness para todos os indivíduos da população
    populacao = funcao_selecao(populacao, fitness) # seleciona-se os indivíduos com os maiores fitness

    # Cruzamento
    # cruzando os indivíduos selecionados
    # esquema intercalado para a escolha dos indivíduos que serão pai e daqueles que serão mãe
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0 # indicando a posição do elemento a ser substituído

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO: # checa se vai rolar cruzamento (dica: chance de 0.5)
            filho1, filho2 = funcao_cruzamento(pai, mae) 
            populacao[contador] = filho1 # filho1 será substituido na posição 0, substituindo o pai
            populacao[contador + 1] = filho2 # filho2 será substituido na posição 1, substituindo a mãe

        contador = contador + 2 # o contador fora do if ajuda a garantir que a substituição vai acontecer, dentro do if haveria uma condição
                                # para não ficar substituindo só no mesmo lugar, adciona-se 2 a cada novo valor de contador
                                # concluindo a lista da nova geração

    # Mutação
    for n in range(len(populacao)): # para cada individuo na população
        if random.random() <= CHANCE_MUTACAO: # lembrando que a chance de acontecer mutação é de 0.05, por isso o if, SE o valor for menos que a constante, rola a mutação
            individuo = populacao[n] # indivíduos são os elementos que ocupam as posições da população (pensando nessa como uma lista, por ex)
            populacao[n] = funcao_mutacao(individuo) # populção vai corresponder aos indivíduos após serem submetidos ao processo de mutação (nem todos eles sofrerão)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness) # lembrando que é um processo de maximização
    posicao = fitness.index(maior_fitness) 
    individuo = populacao[posicao].copy() # se você identifica a posição do maior fitness dentro da população, você consequentemente está identificando o indivíduo
    valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO: # o fitness agora precisa ser maior, pois é maximização
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior valor: {valor} | Peso: {peso}")

# reportando o melhor individuo encontrado
print()
print("Você deve pegar os seguintes itens:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()

valor_total, peso_total = computa_mochila(
    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
)

print(
    f"Com isso, sua mochila terá o valor de {valor_total} dinheiros."
    f"e peso de {peso_total} unidades de massa."
)

Maior valor: 13900 | Peso: 9.1
Maior valor: 15400 | Peso: 12.1
Maior valor: 16900 | Peso: 14.1

Você deve pegar os seguintes itens:
+ Quadrinho super raro do Aranha-Homem da vida real
+ docinhos para o stress
+ energético
+ tablet
+ teclado musical

Com isso, sua mochila terá o valor de 16900 dinheiros.e peso de 14.1 unidades de massa.


<br>

#### DISCUSSÃO

<div style=' text-align: justify; text-justify: inter-word;'>
    O objetivo do presente experimento consistia em encontrar uma resposta que permitisse que uma mochila, com capacidade máxima para aguentar 15 kilos, abrigasse o número de produtos que custasse o maior valor possível, percebe-se, então, desde o início, que se trata de um problema de maximização, mas que ele é limitado por um fator extremamente importante para a resolução do problema: peso. Portanto, nosso código não deve somente considerar a busca pelo maior preço, mas também uma busca pelo maior preço reunindo produtos que o conjunto deles não ultrapasse o peso máximo. Caso contrário, a mochila rasgará e os produtos serão perdidos. Tratamos esse problema como um problema de caixas binárias, apesar de termos um dicionário repleto de opções de produtos. Isso pode ser observado quando em 'if pega_ou_não', definimos 1 para pegar e 0 para não pegar.
    <br>Em primeiro lugar, definimos as nossas constantes: a) TAMANHO_POP: diz respeito ao tamanho da população, definimos que ela será composta por 20 indivíduos no total; 20 indivíduos em todas as gerações. Dito isso, o número total de gerações (NUM_GERACOES) também que queremos criar é 100. Por último, gostaria de destacar as últimas duas constantes 'CHANCE_CRUZAMENTO' e 'CHANCE_MUTACAO', elas refletem uma realidade: nem todos os indivíduos em uma população cruzam gerando para gerar descendentes e nem todos os genes sofrem mutação. As outras constantes são os possíveis objetos que entrarão em uma mochila e uma muito importante: o peso MÁXIMO da mochila, que é a nossa restrição.
    <br>Definidas as constantes e considerando que estamos fazendo uso de algoritmos genéticos, foi necessário aqui ter todo o passo a passo, desde a criação dos indivíduos, que formam a população, que caracteriza uma geração; a seleção de indivíduos que poderão sofrer cruzamento, dando origem aos novos indivíduos da nova geração. Importante lembrar também que ainda observamos a mutação, realizada após o cruzamento, responsável por alterar os genes componentes dos novos indivíduos. Nesse caso, cada indivíduo corresponde a uma composição de objetos da mochila. Algumas peculiaridades do nosso problema se encontra no fato, por exemplo, de que, logo no início, precisamos tranformar o nosso dicionário em uma lista que interesse ao algoritmo genético. Assim, registramos os nomes deles, o que não implica em desconsiderar valores ou peso, essas informações são solicitadas logo na função para definir o fitness.
    <br> Ao rodar o código pela primeira vez, o valor mais alto para o preço da mochila foi de 17 mil e 900 dinheiros, alcançando um peso de 14.1 unidades de massa. Além disso, aparaceram outras duas opções (que não foram escolhidas como resposta):
<br>1. Maior valor: 12900 | Peso: 14.1 = Essa, apesar de ter o mesmo peso que a resposta final, não oferecia o menor preço e o que importa para a escolha da resposta é a combinação dessas duas variáveis;
<br>2. Maior valor: 16400 | Peso: 12.1 = Essa apresenta um valor elevado, mas não era a melhor das opções.
    <br> Ao variar o tamanho da população, por exemplo, não observou-se muitas mudanças, no final continuaram aparecendo 3-4 opções, ainda que a população fosse formada pelo dobro (40) de indivíduos por exemplo, e sempre a melhor dela era a escolhida. Para alterações quanto ao número de gerações, a tendência foi a mesma. Nesse sentido, é útil dizer que o número de opções de resposta só iria variar se não houvesse tanta discrepância entre os valores dos produtos da mochila, as opções ficam meio limitadas nesse caso.
</div>

<br>

> <hr>

## Conclusão
<div style=' text-align: justify; text-justify: inter-word;'>
    O problema instruía que uma mochila fosse preenchida por itens de maneira a apresentar o maior valor possível, contudo, sem ultrapassar a limitação estabelecida logo no início: o peso máximo. Ao rodarmos o código, a resposta é composta por seis itens (6/10), esses que, somados, representam grande valor de dinheiros e que, como desejado, não ultrapassam o peso máximo (15), embora cheguem perto dele. Em suma, ao utilizar algoritmos genéticos para a resolução do experimento, obtivemos um bom resultado respeitando à limitação estabelecida logo no início.
</div>


<HR>

## Playground

